In [1]:
import requests
from requests.structures import CaseInsensitiveDict

url = "https://api.geoapify.com/v1/geocode/search?text=38%20Upper%20Montagu%20Street%2C%20Westminster%20W1H%201LJ%2C%20United%20Kingdom&apiKey=aa6a3fd227ef4d0da2330f2bff2ca817"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

resp = requests.get(url, headers=headers)

print(resp.status_code)

200


races_* columns description:
rid - Race id;
course - Course of the race, country code in brackets, AW means All Weather, no brackets means UK;
time - Time of the race in hh:mm format, London TZ;
date - Date of the race;
title - Title of the race;
rclass - Race class;
band - Band;
ages - Ages allowed
distance - Distance;
condition - Surface condition;
hurdles - Hurdles, their type and amount;
prizes - Places prizes;
winningTime - Best time shown;
prize - Prizes total (sum of prizes column);
metric - Distance in meters;
countryCode - Country of the race;
ncond - condition type (created from condition feature);
class - class type (created from rclass feature).

horses_* columns description:
rid - Race id;
horseName - Horse name;
age - Horse age;
saddle - Saddle # where horse starts;
decimalPrice - 1/Decimal price;
isFav - Was horse favorite before start? Can be more then one fav in a race;
trainerName - Trainer name;
jockeyName - Jockey name;
position - Finishing position, 40 if horse didn't finish;
positionL - how far a horse has finished from the pursued horse, horses corpses;
dist - how far a horse has finished from a winner, horses corpses;
weightSt - Horse weight in St;
weightLb - Horse weight in Lb;
overWeight - Overweight code;
outHandicap - Handicap;
headGear - Head gear code;
RPR - RP Rating;
TR - Topspeed;
OR - Official Rating
father - Horse's Father name;
mother - Horse's Mother name;
gfather - Horse's Grandfather name;
runners - Runners total;
margin - Sum of decimalPrices for the race;
weight - Horse weight in kg;
res_win - Horse won or not;
res_place - Horse placed or not

In [2]:
# Import our dependencies
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf



In [3]:
 # The path to excel file
file = "Resources/latitude_longitude_races.csv"

# Read merged data into pandas
df_lat_lon = pd.read_csv(file)
df_lat_lon.head(5)

,"1. Fakenham (GB): 52.8414° N, 0.8537° E"
0,"2. Cheltenham (GB): 51.9250° N, 2.0620° W"
1,"3. Tramore (IRE): 52.1569° N, 7.1328° W"
2,"4. Fairyhouse (IRE): 53.5025° N, 6.3881° W"
3,"5. Southwell (AW) (GB): 53.0792° N, 0.9458° W"
4,"6. Musselburgh (GB): 55.9442° N, 3.0629° W"


In [4]:
# Split the necessary column into four columns
df_lat_lon[['race', 'country', 'coordinates']] = df_lat_lon[0].str.split(' ', n=2, expand=True)
df_lat_lon[['lat', 'NS', 'long', 'EW']] = df_lat_lon['coordinates'].str.extract('(.+°) ([NS]), (.+°) ([EW])')

# Remove any extra whitespace from the columns
df_lat_lon['race'] = df_lat_lon['race'].str.strip()
df_lat_lon['country'] = df_lat_lon['country'].str.strip()
df_lat_lon['lat'] = df_lat_lon['lat'].str.strip()
df_lat_lon['long'] = df_lat_lon['long'].str.strip()

# Remove the unnecessary columns
df_lat_lon.drop([0, 'coordinates', 'NS', 'EW'], axis=1, inplace=True)

# Reset the index
df_lat_lon.reset_index(drop=True, inplace=True)

# Print the DataFrame
print(df_lat_lon)






KeyError: 0

**Data Cleanup - information from Kaggle**

In [71]:
 # The path to SV file
file = "Resources/merged_data.csv"

# Read merged data into pandas
df = pd.read_csv(file)
df.head(5)

C:\Users\aumek\AppData\Local\Temp\ipykernel_11684\1305083634.py:5: DtypeWarning: Columns (11,19,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


,Unnamed: 0,rid,course,time,date,title,rclass,band,ages,distance,...,OR,father,mother,gfather,runners,margin,weight,res_win,res_place,price
0,0,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,NaN,Pour Moi,Laughing Water,Duke Of Marmalade,9.0,1.199095,66.0,1.0,1.0,NaN
1,1,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,110.0,Getaway,Shouette,Sadler's Wells,9.0,1.199095,65.0,0.0,1.0,NaN
2,2,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,NaN,Vale Of York,Finnmark,Halling,9.0,1.199095,72.0,0.0,1.0,NaN
3,3,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,NaN,Sixties Icon,Highland Jig,Norse Dancer,9.0,1.199095,72.0,0.0,0.0,NaN
4,4,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,NaN,Iffraaj,Hallowed Park,Barathea,9.0,1.199095,72.0,0.0,0.0,NaN


In [134]:
 # Get a list of all of our columns for easy reference
df.columns

Index(['Unnamed: 0', 'rid', 'course', 'time', 'date', 'title', 'rclass',
       'band', 'ages', 'distance', 'condition', 'hurdles', 'prizes',
       'winningTime', 'prize', 'metric', 'countryCode', 'ncond', 'class',
       'currency', 'horseName', 'age', 'saddle', 'decimalPrice', 'isFav',
       'trainerName', 'jockeyName', 'position', 'positionL', 'dist',
       'weightSt', 'weightLb', 'overWeight', 'outHandicap', 'headGear', 'RPR',
       'TR', 'OR', 'father', 'mother', 'gfather', 'runners', 'margin',
       'weight', 'res_win', 'res_place', 'price'],
      dtype='object')

In [135]:
import numpy as np

# Replace empty strings with NaN values
df.replace('', np.nan, inplace=True)

# Fill NaN values with desired values
df['age'].fillna(mean_value['age'], inplace=True)
df['position'].fillna(mean_value['position'], inplace=True)
df['positionL'].fillna(mean_value['positionL'], inplace=True)
df['distance'].fillna(mean_value['distance'], inplace=True)
df['outHandicap'].fillna(mean_value['outHandicap'], inplace=True)
df['RPR'].fillna(mean_value['RPR'], inplace=True)


**Preparing the data For Model1 - where we are predicting if the 'favorite' horse wins, we used two datasets from "Kaggle" as listed below:**

*dataset 1 = races_* columns description: 
- rid - Race id; 
- Distance
- winningTime - Best time shown
- metric - Distance in meters
- class - class type (created from rclass feature).

*dataset 2 = horses_* columns description: 
- rid - Race id (column used to merge the 2 datasets)
- age - Horse age; 
- saddle - Saddle # where horse starts; 
- isFav - Was horse favorite before start? Can be more then one fav in a race;  
- position - Finishing position, 40 if horse didn't finish; 
- positionL - how far a horse has finished from the pursued horse, horses corpses; 
- outHandicap - Handicap; 
- RPR - RP Rating; 
- TR - Topspeed; 
- OR - Official Rating father - Horse's Father name; 
- runners - Runners total; 
- weight - Horse weight in kg; 
- res_win - Horse won or not; 


In [136]:
# Drop columns
clean_df = df.loc[:, [ 'rid', 'winningTime', 'metric', 'ncond', 'class',
       'age', 'decimalPrice', 'isFav','position', 'positionL', 'distance','outHandicap', 'RPR','saddle',
       'TR', 'OR', 'runners', 'weight', 'res_win']]
clean_df.head(5)

,rid,winningTime,metric,ncond,class,age,decimalPrice,isFav,position,positionL,distance,outHandicap,RPR,saddle,TR,OR,runners,weight,res_win
0,10312,253.88,3218.0,10,4,4,0.076923,0.0,1,4.57028,2m,2.541359,103.0,9.0,103.0,74.677073,9.0,66.0,1.0
1,10312,253.88,3218.0,10,4,6,0.444444,1.0,2,15,2m,2.541359,96.0,7.0,89.0,110.000000,9.0,65.0,0.0
2,10312,253.88,3218.0,10,4,7,0.019608,0.0,3,5.5,2m,2.541359,97.0,5.0,88.0,74.677073,9.0,72.0,0.0
3,10312,253.88,3218.0,10,4,6,0.230769,0.0,4,10,2m,2.541359,89.0,4.0,75.0,74.677073,9.0,72.0,0.0
4,10312,253.88,3218.0,10,4,7,0.038462,0.0,5,5.5,2m,2.541359,82.0,6.0,67.0,74.677073,9.0,72.0,0.0


In [137]:
import re

#remove special characters and normalize distance to show all as 'meters'

# Convert 'dist' column to string data type
clean_df['distance'] = clean_df['distance'].astype(str)

# Define the function to clean the distance values
def clean_distance(distance):
    if re.search(r'\d+m', distance):
        meters = re.findall(r'\d+', distance)
        return int(meters[0])
    elif re.search(r'\d+f', distance):
        furlongs = re.findall(r'\d+', distance)
        meters = int(furlongs[0]) * 201
        return meters
    elif re.search(r'\dm\d+f', distance):
        meters = re.findall(r'\d+m', distance)
        furlongs = re.findall(r'\d+f', distance)
        meters = int(meters[0].strip('m'))
        furlongs = int(furlongs[0].strip('f')) * 201
        return meters + furlongs
    else:
        return distance

# Apply the clean_distance function to the 'dist' column
clean_df['distance'] = clean_df['distance'].apply(clean_distance)

# Print the cleaned DataFrame
clean_df.head(5)


,rid,winningTime,metric,ncond,class,age,decimalPrice,isFav,position,positionL,distance,outHandicap,RPR,saddle,TR,OR,runners,weight,res_win
0,10312,253.88,3218.0,10,4,4,0.076923,0.0,1,4.57028,2,2.541359,103.0,9.0,103.0,74.677073,9.0,66.0,1.0
1,10312,253.88,3218.0,10,4,6,0.444444,1.0,2,15,2,2.541359,96.0,7.0,89.0,110.000000,9.0,65.0,0.0
2,10312,253.88,3218.0,10,4,7,0.019608,0.0,3,5.5,2,2.541359,97.0,5.0,88.0,74.677073,9.0,72.0,0.0
3,10312,253.88,3218.0,10,4,6,0.230769,0.0,4,10,2,2.541359,89.0,4.0,75.0,74.677073,9.0,72.0,0.0
4,10312,253.88,3218.0,10,4,7,0.038462,0.0,5,5.5,2,2.541359,82.0,6.0,67.0,74.677073,9.0,72.0,0.0


In [138]:
#to verify the data type
clean_df.dtypes

rid               int64
winningTime     float64
metric          float64
ncond             int64
class             int64
age               int32
decimalPrice    float64
isFav           float64
position          int32
positionL        object
distance         object
outHandicap     float64
RPR             float64
saddle          float64
TR              float64
OR              float64
runners         float64
weight          float64
res_win         float64
dtype: object

In [139]:
#changing datatype for 'positionL' and 'dist' to numeric
clean_df['positionL'] = pd.to_numeric(clean_df['positionL'], errors='coerce')
clean_df['distance'] = pd.to_numeric(clean_df['distance'], errors='coerce')

# Verify the updated data types
print(clean_df.dtypes)


rid               int64
winningTime     float64
metric          float64
ncond             int64
class             int64
age               int32
decimalPrice    float64
isFav           float64
position          int32
positionL       float64
distance        float64
outHandicap     float64
RPR             float64
saddle          float64
TR              float64
OR              float64
runners         float64
weight          float64
res_win         float64
dtype: object


In [143]:
# Specify the columns you want to drop NA values
columns_to_drop_na = ['isFav', 'res_win']

# Specify the values representing blanks in those columns
blank_values = ['', ' ', '-']

# Replace specific blank values with NaN
clean_df.replace(blank_values, np.nan, inplace=True)

# Drop rows with NA values in the specified columns
clean_df.dropna(subset=columns_to_drop_na, inplace=True)

# Reset the index
clean_df.reset_index(drop=True, inplace=True)


In [145]:
# save to csv to visualize data
clean_df.to_csv('Resources/clean_data_model1.csv', index=False)


In [146]:
# Count the remaining rows and columns in the DataFrame
num_rows, num_columns = clean_df.shape

print("Number of rows:", num_rows)
print("Number of columns:", num_columns)



Number of rows: 149513
Number of columns: 19


In [147]:
y = clean_df["isFav"]
X = clean_df.drop(columns="isFav")

In [148]:
 from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(112134, 18)

In [149]:
 from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)
classifier

LogisticRegression(max_iter=200, random_state=1)

In [150]:
# Convert categorical data to numeric with `pd.get_dummies`
coverted_df= pd.get_dummies(clean_df)
coverted_df

,rid,winningTime,metric,ncond,class,age,decimalPrice,isFav,position,positionL,distance,outHandicap,RPR,saddle,TR,OR,runners,weight,res_win
0,10312,253.88,3218.0,10,4,4,0.076923,0.0,1,4.57028,2.0,2.541359,103.000000,9.0,103.0,74.677073,9.0,66.0,1.0
1,10312,253.88,3218.0,10,4,6,0.444444,1.0,2,15.00000,2.0,2.541359,96.000000,7.0,89.0,110.000000,9.0,65.0,0.0
2,10312,253.88,3218.0,10,4,7,0.019608,0.0,3,5.50000,2.0,2.541359,97.000000,5.0,88.0,74.677073,9.0,72.0,0.0
3,10312,253.88,3218.0,10,4,6,0.230769,0.0,4,10.00000,2.0,2.541359,89.000000,4.0,75.0,74.677073,9.0,72.0,0.0
4,10312,253.88,3218.0,10,4,7,0.038462,0.0,5,5.50000,2.0,2.541359,82.000000,6.0,67.0,74.677073,9.0,72.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149508,415090,99.66,1609.0,5,0,8,0.030303,0.0,7,NaN,1.0,2.541359,74.677073,6.0,NaN,74.677073,11.0,58.0,0.0
149509,415090,99.66,1609.0,5,0,4,0.153846,0.0,8,0.50000,1.0,2.541359,74.677073,11.0,NaN,74.677073,11.0,57.0,0.0
149510,415090,99.66,1609.0,5,0,5,0.044053,0.0,9,NaN,1.0,2.541359,74.677073,8.0,NaN,74.677073,11.0,58.0,0.0
149511,415090,99.66,1609.0,5,0,8,0.041152,0.0,10,1.00000,1.0,2.541359,74.677073,3.0,NaN,74.677073,11.0,58.0,0.0


In [151]:

# Split our preprocessed data into our features and target arrays
#from sklearn.model_selection import train_test_split
sample_size = int(len(coverted_df) * 0.5)  # 50% of the original dataset as I run out of RAM
sample_df = coverted_df[:sample_size]


# Split the preprocessed data into features and target arrays
X = sample_df.drop('isFav', axis=1).values
y = sample_df['isFav'].values


# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)